In [3]:
import yfinance as yf
import numpy as np
from sklearn.preprocessing import MinMaxScaler


dat = yf.Ticker('MSFT')
history_df = dat.history()

In [4]:
def create_sequences(data, seq_length):
    X = []
    y = []
    for i in range(len(data) - seq_length):
        seq = data[i:i + seq_length]
        target = data[i + seq_length]
        X.append(seq)
        y.append(target)
    return np.array(X), np.array(y)

In [5]:
closing_prices = history_df['Close'].values
SEQ_LENGTH = 3
X, y = create_sequences(closing_prices, SEQ_LENGTH)

In [6]:
# Ensure split is chronological
train_size = int(len(X) * 0.8)
val_size = int(len(X) * 0.1)

# Create training, validation, and testing sets
X_train, y_train = X[:train_size], y[train_size]
X_valid, y_valid = X[train_size: train_size + val_size], y[train_size: train_size + val_size]
X_test, y_test = X[train_size + val_size:], y[train_size + val_size:]

In [8]:
# Handle scaling
scaler_X = MinMaxScaler()

X_train_reshaped = X_train.reshape(-1, X_train.shape[-1])